In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
class Layer():
    def __init__(self,size,nextSize):
        self.biases = None
        self.size=size
        self.neurons= np.empty(shape=size)
        self.biases = np.empty(shape=size)
        self.weights = np.empty(shape=(size, nextSize))


In [ ]:
import random

import copy
import  numpy as np
class NeuralNetwork():

    def __init__(self,learningRate,activation,derivative,sizes):
        self.learningRate = learningRate
        self.activation = activation
        self.derivative = derivative
        self.layers=[]
        for i in range(0,len(sizes)):
            nextSize = 0
            if(i<=len(sizes)-1):
                nextSize=sizes[i]
                self.layers.append(Layer(sizes[i],nextSize))
                for j in range(0,sizes[i]):
                    self.layers[i].biases[j]=random.random()* 2.0 - 1.0
                    for k in range(0,nextSize):
                        self.layers[i].weights[j][k]=random.random()*2.0-1.0




    def feedForward(self,inputs):
        self.layers[0].neurons=copy.deepcopy(inputs)
        for i in range(1,len(self.layers)):
            l = self.layers[i-1]
            l1=self.layers[i]
            l1.neurons = np.dot(l.neurons, l.weights)[:l1.size]
            l1.neurons = self.activation(l1.neurons)
        return  self.layers[-1].neurons

    def backpropogation(self,targets):
        errors=np.zeros(shape=(1,self.layers[-1].size))
        errors[0]= targets - self.layers[-1].neurons
        # for i in range(0,self.layers[-1].size):
        #     errors[0][i]=targets[i]-self.layers[-1].neurons[i]
        for k in range(len(self.layers)-2,0,-1):
            l=self.layers[k]
            l1=self.layers[k+1]
            errorsNext=np.zeros(shape=(1,l.size))
            gradients=np.zeros(shape=(1,l.size))
            tmp1 = self.derivative(self.layers[k + 1].neurons)
            tmp2=errors[0]*tmp1
            gradients[0][:l1.size]=tmp2*self.learningRate
            a=gradients.reshape(gradients.shape[1],1)

            b=l.neurons.reshape(1,l.neurons.shape[0])
            tmp = np.dot(a,b)
            deltas = np.empty(shape=(l1.size,l.size))
            deltas =tmp[:l1.size]

            for i in range(0, l.size):
                errorsNext[0][i]= np.dot(l.weights[i][:l1.size],errors[0])


            errors=copy.deepcopy(errorsNext)
            weightsNew=np.empty(shape=(len(l.weights),len(l.weights[0])))

            A = deltas.T
            temp=l.weights[:,:l1.size]
            weightsNew=temp+A
            # for i in range(0, l1.size):
            #     for j in range(0, l.size):
            #         weightsNew[j][i]=l.weights[j][i]+deltas[i][j]



            # a=l.weights[:l.size][:l1.size]
            # b=deltas.reshape(l1.size,l.size)
            # tmp=a+b
            l.weights=weightsNew
            for i in range(0, l1.size):
                l1.biases[i]+=gradients[0][i]



In [ ]:
directoty = "./drive/My Drive/nn/"

In [ ]:

def sigmoid( x):
    return 1 / (1.0 + np.exp(-x))

def dsigmoid(x):
        return x * (1 - x)



def read_Data(type):
    dataset=type
    path=directoty
    if dataset == "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
    else:
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    Y = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    X = pyarray("B", fimg.read())
    fimg.close()
    return X,Y,rows,cols,size

In [ ]:


import numpy as np
import random
import os
import struct
from array import array as pyarray

nn=NeuralNetwork(0.023,sigmoid,dsigmoid,[784,512,128,10])#,512,128,32,10])
train_X, train_Y, rows, cols, size = read_Data("training")
train_X = train_X[:int(len(train_X) )]
train_Y = train_Y[:int(len(train_Y))]

images = []

for i in range(0, len(train_Y)):
    images.append(np.array(train_X[i * rows * cols: (i + 1) * rows * cols]).reshape((rows, cols)))
images = np.array(images)
p = np.reshape(images, (len(images), rows * cols))
p=p*(0.99/255)
print(p.shape)
labels = np.array(train_Y)
labels = labels.reshape(labels.shape[0], 1)

errors=[]
indexs=[]
samples = p.shape[0]
samples=2000
k=0
for i in range(100):
    right=1
    errorSum=0.0
    batchsize=200
    for j in range(0,samples):
        imgIndex=int(random.random()*samples)
        targets=np.zeros(shape=10)
        digit =labels[imgIndex][0]
        targets[digit]=1
        outputs = nn.feedForward(p[imgIndex])
        maxDigit=0
        maxDigitWeight=-1

        if(digit==np.argmax(outputs)):
          right+=1
        for k in range(10):
            errorSum+=(targets[k]-outputs[k])*(targets[k]-outputs[k])/2
            errors.append(errorSum)
            indexs.append(k)
            k+=1
        nn.backpropogation(targets)

    print("epoch: " + str(i) + ". correct: " + str(right) + ". error: " + str(errorSum))

print(nn.layers[0].weights)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(errors)
plt.plot(errors)

In [ ]:
batchsize=100
rigth_t=0
for j in range(0,batchsize):
    outputs_t = nn.feedForward(p[j])
    digit_t =labels[j][0]
    if(digit_t==np.argmax(outputs_t)):
          rigth_t+=1

print(rigth_t)

In [ ]:

train_X, train_Y, rows, cols, size = read_Data("test")
train_X = train_X[:int(len(train_X) )]
train_Y = train_Y[:int(len(train_Y))]

images = []

for i in range(0, len(train_Y)):
    images.append(np.array(train_X[i * rows * cols: (i + 1) * rows * cols]).reshape((rows, cols)))
images = np.array(images)
p = np.reshape(images, (len(images), rows * cols))
p=p*(0.99/255)
print(p.shape)
labels = np.array(train_Y)
labels = labels.reshape(labels.shape[0], 1)


rigth_t=0
for i in range(p.shape[0]):

  digit_t =labels[i][0]
  outputs_t = nn.feedForward(p[i])
  maxDigit=0
  maxDigitWeight=-1
  if(digit_t==np.argmax(outputs_t)):
          rigth_t+=1

print(rigth_t)
  
